# Creation Dataset

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import shutil

cartella_immagini = '/content/drive/MyDrive/CNN-CNR/Data/roi/D20240912T092716_IFCB182'
file_excel = '/content/drive/MyDrive/CNN-CNR/Data/excel/D20240912T092716_IFCB182.xlsx'
cartella_destinazione = '/content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented'
file_excel_classi = '/content/drive/MyDrive/CNN-CNR/Data/classes.xlsx'

def smista_immagini_per_classe(cartella_immagini, file_excel, file_excel_classi, cartella_destinazione):
    # Leggi il file Excel principale
    df = pd.read_excel(file_excel)

    # Stampa le colonne per verifica
    print("Colonne trovate nel file Excel principale:", df.columns)

    # Leggi il file Excel delle classi
    df_classi = pd.read_excel(file_excel_classi, header=None)

    # Controlla se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(cartella_destinazione):
        os.makedirs(cartella_destinazione)

    for index, row in df.iterrows():
        codice_immagine = int(row['roi number'])
        codice_classe = row['manual_classification']

        # Gestisci il caso di classificazione mancante
        if pd.isna(codice_classe):
            print(f"Classificazione mancante (NaN) per il codice immagine {codice_immagine}. Riga saltata.")
            continue

        # Converti il codice classe in stringa (gestione dei numeri float e int)
        try:
            codice_classe_int = int(codice_classe)
        except ValueError:
            print(f"Codice classe non valido: {codice_classe}. Riga saltata.")
            continue

        # Trova il nome della classe dal file delle classi
        try:
            nome_classe = df_classi.loc[codice_classe_int - 1, 0].strip()
            print(f"Classe corrispondente al codice {codice_classe_int}: {nome_classe}")
        except (KeyError, IndexError):
            print(f"Codice classe {codice_classe_int} non trovato nel file delle classi. Riga saltata.")
            continue

        # Crea o utilizza il nome della cartella con il nome della classe
        cartella_classe = cartella_destinazione+'/'+str(nome_classe)
        print(f"Cartella della classe: {cartella_classe}")

        # Assicura che la cartella della classe esista (senza duplicati)
        os.makedirs(cartella_classe, exist_ok=True)

        # Trova l'immagine corrispondente
        codice_immagine_str = str(codice_immagine).zfill(5)


        immagine_trovata = None
        for file in os.listdir(cartella_immagini):
            if file.endswith(codice_immagine_str + ".png"):
                immagine_trovata = file
                break



        # Sposta l'immagine nella cartella corretta
        if immagine_trovata:
            sorgente = os.path.join(cartella_immagini, immagine_trovata)
            destinazione = os.path.join(cartella_classe, immagine_trovata)

            # Controlla se l'immagine esiste già nella cartella di destinazione
            if not os.path.exists(destinazione):
                shutil.move(sorgente, destinazione)
                print(f"Immagine {immagine_trovata} spostata in {cartella_classe}.")
            else:
                print(f"Immagine {immagine_trovata} già esistente in {cartella_classe}.")
        else:
            print(f"Immagine corrispondente a {codice_immagine_str} non trovata.")

    print("Esecuzione terminata con successo.")

#smista_immagini_per_classe(cartella_immagini, file_excel, file_excel_classi, cartella_destinazione)



Colonne trovate nel file Excel principale: Index(['roi number', 'manual_classification'], dtype='object')
Classe corrispondente al codice 1: unclassified
Cartella della classe: /content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented/unclassified
Immagine D20240912T092716_IFCB182_00001.png spostata in /content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented/unclassified.
Classe corrispondente al codice 76: Chaetoceros diversus
Cartella della classe: /content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented/Chaetoceros diversus
Immagine D20240912T092716_IFCB182_00002.png spostata in /content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented/Chaetoceros diversus.
Classe corrispondente al codice 1: unclassified
Cartella della classe: /content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented/unclassified
Immagine D20240912T092716_IFCB182_00003.png spostata in /content/drive/MyDrive/CNN-CNR/Datasets/ORIGINAL/original-augmented/unclassified.

In [ ]:
import os

def elimina_immagini_con_rot(cartella_principale):
    """
    Elimina tutte le immagini che contengono 'rot' nel nome all'interno della cartella specificata e delle sue sottocartelle.

    :param cartella_principale: Percorso della cartella principale
    """
    # Definiamo le estensioni delle immagini da considerare
    estensioni_immagini = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}

    # Attraversa tutte le cartelle e sottocartelle
    for root, dirs, files in os.walk(cartella_principale):
        for file in files:
            # Estrai l'estensione del file
            _, estensione = os.path.splitext(file)
            if estensione.lower() in estensioni_immagini:
                # Controlla se 'rot' è nel nome del file (case insensitive)
                if 'rot' in file.lower():
                    percorso_file = os.path.join(root, file)
                    try:
                        os.remove(percorso_file)
                        print(f"Eliminato: {percorso_file}")
                    except Exception as e:
                        print(f"Errore nell'eliminare {percorso_file}: {e}")


#elimina_immagini_con_rot('/content/drive/MyDrive/CNN-CNR/Datasets/Dataset-plancton-cnr-augmented')


## Duplicate Dataset

In [2]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm

def copia_contenuti_cartella_new_version(cartella_sorgente, cartella_destinazione):
    """
    Copies all contents (files and subfolders) from the source folder to the destination folder.

    :param cartella_sorgente: Path to the source folder
    :param cartella_destinazione: Path to the destination folder
    """
    # Convert paths to Path objects for easier management
    sorgente = Path(cartella_sorgente)
    destinazione = Path(cartella_destinazione)

    if not sorgente.exists():
        raise FileNotFoundError(f"The source folder '{sorgente}' does not exist.")
    if not sorgente.is_dir():
        raise NotADirectoryError(f"The source '{sorgente}' is not a folder.")

    # Create the destination folder if it does not exist
    destinazione.mkdir(parents=True, exist_ok=True)

    # Initialize counters for the total number of files copied and directories created
    total_files_copied = 0
    total_dirs_created = 0

    # Gather all files to be copied
    files_to_copy = []
    for root, dirs, files in os.walk(sorgente):
        root_path = Path(root)
        relative_path = root_path.relative_to(sorgente)
        dest_path = destinazione / relative_path

        # Ensure all subdirectories exist in the destination
        for dir_name in dirs:
            dir_dest = dest_path / dir_name
            try:
                dir_dest.mkdir(parents=True, exist_ok=True)
                total_dirs_created += 1  # Increment the counter for each directory created
            except Exception as e:
                print(f"Error creating directory {dir_dest}: {e}")

        # Add files to the list with their source and destination paths
        for file_name in files:
            file_sorgente = root_path / file_name
            file_destinazione = dest_path / file_name
            files_to_copy.append((file_sorgente, file_destinazione))

    # Initialize tqdm progress bar with the total number of files
    with tqdm(total=len(files_to_copy), desc="Copying files", unit="file") as pbar:
        for file_sorgente, file_destinazione in files_to_copy:
            try:
                shutil.copy2(file_sorgente, file_destinazione)
                total_files_copied += 1  # Increment the counter for each successful copy
            except Exception as e:
                print(f"Error copying {file_sorgente} to {file_destinazione}: {e}")
            finally:
                pbar.update(1)  # Update the progress bar

    # Print the total number of directories created and files copied
    print(f"\nTotal directories created: {total_dirs_created}")
    print(f"Total files copied: {total_files_copied}")


In [3]:

# Definisci i percorsi della cartella sorgente e di destinazione
cartella_sorgente = '/content/drive/MyDrive/CNN-CNR/backup/final-15-backup'
cartella_destinazione = '/content/drive/MyDrive/CNN-CNR/Datasets/final-200-v2'


copia_contenuti_cartella_new_version(cartella_sorgente, cartella_destinazione)

Copying files: 100%|██████████| 2216/2216 [07:56<00:00,  4.65file/s]


Total directories created: 25
Total files copied: 2216


------------------------------

In [ ]:
import os
from pathlib import Path
from PIL import Image

def trova_dimensioni_estreme_e_media(cartella_principale):
    """
    Trova le dimensioni (altezza x larghezza) dell'immagine più grande e di quella più piccola,
    e calcola la media delle dimensioni delle immagini all'interno della cartella principale
    e delle sue sottocartelle.

    :param cartella_principale: Percorso della cartella principale
    :return: Tuple contenente (dim_piccola, dim_grande, media_altezza, media_larghezza)
    """
    # Definiamo le estensioni delle immagini da considerare
    estensioni_immagini = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp', '.heic'}

    # Convertiamo il percorso in un oggetto Path
    path_principale = Path(cartella_principale)

    if not path_principale.exists():
        raise FileNotFoundError(f"La cartella principale '{cartella_principale}' non esiste.")
    if not path_principale.is_dir():
        raise NotADirectoryError(f"Il percorso '{cartella_principale}' non è una cartella.")

    # Inizializziamo le variabili per tenere traccia delle dimensioni
    immagine_piu_piccola = None
    immagine_piu_grande = None
    area_piccola = None
    area_grande = None

    # Variabili per il calcolo della media
    somma_altezze = 0
    somma_larghezze = 0
    conteggio_immagini = 0

    # Attraversiamo ricorsivamente tutte le directory e i file
    for file in path_principale.rglob('*'):
        if file.is_file() and file.suffix.lower() in estensioni_immagini:
            try:
                with Image.open(file) as img:
                    width, height = img.size
                    area = width * height

                    # Aggiorna l'immagine più piccola
                    if area_piccola is None or area < area_piccola:
                        immagine_piu_piccola = (height, width)
                        area_piccola = area

                    # Aggiorna l'immagine più grande
                    if area_grande is None or area > area_grande:
                        immagine_piu_grande = (height, width)
                        area_grande = area

                    # Aggiorna le somme per il calcolo della media
                    somma_altezze += height
                    somma_larghezze += width
                    conteggio_immagini += 1

            except Exception as e:
                print(f"Errore nell'aprire l'immagine {file}: {e}")

    if conteggio_immagini == 0:
        print("Nessuna immagine valida trovata nella cartella specificata.")
        return None, None, None, None

    # Calcola le medie
    media_altezza = somma_altezze / conteggio_immagini
    media_larghezza = somma_larghezze / conteggio_immagini

    return immagine_piu_piccola, immagine_piu_grande, media_altezza, media_larghezza

In [ ]:


# Inserisci il percorso della cartella principale
cartella = '/content/drive/MyDrive/CNN-CNR/Datasets/original-augmented'

try:
    piccola, grande, media_alt, media_larg = trova_dimensioni_estreme_e_media(cartella)
    if piccola and grande:
        print(f"Immagine più piccola: Altezza = {piccola[0]} px, Larghezza = {piccola[1]} px")
        print(f"Immagine più grande: Altezza = {grande[0]} px, Larghezza = {grande[1]} px")
        print(f"Media delle altezze: {media_alt:.2f} px")
        print(f"Media delle larghezze: {media_larg:.2f} px")
    else:
        print("Non sono state trovate immagini valide.")
except Exception as e:
    print(f"Errore: {e}")



Errore nell'aprire l'immagine /content/drive/MyDrive/CNN-CNR/Datasets/original-augmented/Gymnodinium spp. <20/D20210810T034341_IFCB114_03365.png: cannot identify image file '/content/drive/MyDrive/CNN-CNR/Datasets/original-augmented/Gymnodinium spp. <20/D20210810T034341_IFCB114_03365.png'
Immagine più piccola: Altezza = 224 px, Larghezza = 224 px
Immagine più grande: Altezza = 224 px, Larghezza = 224 px
Media delle altezze: 224.00 px
Media delle larghezze: 224.00 px


In [ ]:
import os
import pandas as pd


def check_missing_classes(dataset_path, excel_path, sheet_name=0):
    # Leggi i nomi delle classi dal file Excel e rinomina la prima colonna
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)
    class_names_from_excel = set(df[0].dropna().unique())

    # Prendi i nomi delle cartelle presenti nel dataset
    class_names_in_dataset = set(
        folder_name for folder_name in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder_name))
    )

    # Calcola i nomi mancanti e ordina in ordine alfabetico
    missing_classes = sorted(class_names_from_excel - class_names_in_dataset)

    # Numero di classi presenti e mancanti
    num_present_classes = len(class_names_in_dataset)
    num_missing_classes = len(missing_classes)

    return {
        "missing_classes": missing_classes,
        "num_present_classes": num_present_classes,
        "num_missing_classes": num_missing_classes
    }



In [ ]:
dataset_path = '/content/drive/MyDrive/CNN-CNR/backup/Dataset-plancton-cnr-original'

excel_path = "/content/drive/MyDrive/CNN-CNR/Data/excel/D20240722T120142_IFCB182.xlsx"

result = check_missing_classes(dataset_path, excel_path)

#print("Classi mancanti:", result["missing_classes"])
print("Numero di classi presenti:", result["num_present_classes"])
print("Numero di classi mancanti:", result["num_missing_classes"])
print("Lista classi mancanti: ")
for elem in result["missing_classes"]:
  print(elem)


### Rename image for each class

In [ ]:
import os

def rename_images_by_class(dataset_path):
    """
    Renames images in a dataset by prefixing the filename with the class name (folder name)
    and an incrementing counter.

    Args:
        dataset_path (str): Path to the dataset. The dataset is expected to be organized into
                            subfolders, where each subfolder represents a class and contains images.
    """
    # Ensure the dataset path exists
    if not os.path.exists(dataset_path):
        print(f"Error: Dataset path '{dataset_path}' does not exist.")
        return

    # Iterate over each folder (class) in the dataset path
    for class_folder in os.listdir(dataset_path):
        class_folder_path = os.path.join(dataset_path, class_folder)

        # Ensure the path is a directory (skip files)
        if os.path.isdir(class_folder_path):
            print(f"Processing class: {class_folder}")

            # Initialize a counter for images in the current class folder
            counter = 1

            # Iterate over each file in the class folder
            for file_name in os.listdir(class_folder_path):
                file_path = os.path.join(class_folder_path, file_name)

                # Ensure the file is an image (optional: check file extension)
                if os.path.isfile(file_path):
                    # Extract file extension (e.g., .jpg, .png)
                    file_extension = os.path.splitext(file_name)[1]

                    # Create the new filename: className_1, className_2, etc.
                    new_file_name = f"{class_folder}_{counter}{file_extension}"
                    new_file_path = os.path.join(class_folder_path, new_file_name)

                    # Rename the file
                    os.rename(file_path, new_file_path)
                    print(f"Renamed '{file_name}' to '{new_file_name}'")

                    # Increment the counter
                    counter += 1

    print("Renaming completed!")

# Example usage:
# rename_images_by_class("path/to/dataset")
